In [28]:
# library imports

import os
import pandas as pd

In [29]:
# reading, processing and merging files

folders = os.listdir('./raw_weather_data')

dataset = pd.DataFrame()

for folder in folders:

    files = os.listdir(f'./raw_weather_data/{folder}')

    for file in files:

        header = pd.read_csv(f'./raw_weather_data/{folder}/{file}', encoding='latin-1', delimiter=';', nrows=7).transpose().reset_index()
        header.columns = header.iloc[0]
        header = header.iloc[1:]

        data = pd.read_csv(f'./raw_weather_data/{folder}/{file}', encoding='latin-1', delimiter=';', header=8).replace(',', '.', regex=True)
        data = data.drop(columns=[column for column in data.columns if 'Unnamed' in column])
        data.columns = [column.replace(' (YYYY-MM-DD)', '').upper() for column in data.columns]
        data.iloc[:, 2:] = data.iloc[:, 2:].astype(float)

        def min_value(column):
            return column.iloc[1:].min()

        def max_value(column):
            return column.iloc[1:].max()
        
        data = data.groupby('DATA').agg({
            'PRECIPITAÇÃO TOTAL, HORÁRIO (MM)': 'sum',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)': 'mean',
            'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (MB)': max_value,
            'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (MB)': min_value,
            'RADIACAO GLOBAL (KJ/M²)': 'mean',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'mean',
            'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'mean',
            'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': max_value,
            'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': min_value,
            'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': max_value,
            'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': min_value,
            'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': max_value,
            'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': min_value,
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'mean',
            'VENTO, RAJADA MAXIMA (M/S)': max_value,
            'VENTO, VELOCIDADE HORARIA (M/S)': 'mean'
        }).reset_index()

        data = pd.merge(header, data, how='cross')

        dataset = pd.concat([dataset, data], ignore_index=True)

In [45]:
# fixing columns

dataset['LATITUDE'] = dataset['LATITUDE'].str.replace(',', '.').astype(float)
dataset['LONGITUDE'] = dataset['LONGITUDE'].str.replace(',', '.').astype(float)
dataset['ALTITUDE'] = dataset['ALTITUDE'].str.replace(',', '.').str.replace('F', '0').astype(float)

dataset['REGIÃO:'] = dataset['REGIÃO:'].fillna(dataset['REGI?O:'])
dataset['ESTAÇÃO:'] = dataset['ESTAÇÃO:'].fillna(dataset['ESTAC?O:'])
dataset['DATA DE FUNDAÇÃO (YYYY-MM-DD):'] = dataset['DATA DE FUNDAÇÃO (YYYY-MM-DD):'].fillna(dataset['DATA DE FUNDAC?O:'])

dataset['REGIÃO:'] = dataset['REGIÃO:'].fillna(dataset['REGIAO:'])
dataset['ESTAÇÃO:'] = dataset['ESTAÇÃO:'].fillna(dataset['ESTACAO:'])
dataset['DATA DE FUNDAÇÃO (YYYY-MM-DD):'] = dataset['DATA DE FUNDAÇÃO (YYYY-MM-DD):'].fillna(dataset['DATA DE FUNDACAO:'])

dataset = dataset.drop(columns=['REGI?O:', 'ESTAC?O:', 'DATA DE FUNDAC?O:', 'REGIAO:', 'ESTACAO:', 'DATA DE FUNDACAO:'])

dataset.columns = [column.replace(':', '').replace(' (YYYY-MM-DD)', '').replace('MM', 'mm').replace('MB', 'mB').replace('KJ/M', 'kJ/m').replace('M/S', 'm/s') for column in dataset.columns]

In [35]:
# fixing date columns

dataset['DATA'] = pd.to_datetime(dataset['DATA'].str.replace('-', '/'), format='%Y/%m/%d')

dataset['DATA DE FUNDAÇÃO'] = dataset['DATA DE FUNDAÇÃO'].apply(
    lambda x: pd.to_datetime(x, format='%d/%m/%y') if '/' in x else pd.to_datetime(x, format='%Y-%m-%d')
)

In [46]:
# visualizing dataset

dataset

,REGIÃO,UF,ESTAÇÃO,CODIGO (WMO),LATITUDE,LONGITUDE,ALTITUDE,DATA DE FUNDAÇÃO,DATA,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",...,TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
0,CO,DF,BRASILIA,A001,-15.789444,-47.925833,1159.54,2000-05-07,2014-01-01,0.0,...,17.558333,29.2,18.3,19.6,13.8,98.0,39.0,74.541667,6.9,1.558333
1,CO,DF,BRASILIA,A001,-15.789444,-47.925833,1159.54,2000-05-07,2014-01-02,0.0,...,16.666667,30.1,18.2,19.6,11.6,95.0,35.0,68.291667,7.6,2.095833
2,CO,DF,BRASILIA,A001,-15.789444,-47.925833,1159.54,2000-05-07,2014-01-03,0.0,...,14.408333,29.7,17.4,18.4,6.4,96.0,24.0,58.500000,7.9,2.000000
3,CO,DF,BRASILIA,A001,-15.789444,-47.925833,1159.54,2000-05-07,2014-01-04,0.0,...,11.758333,29.8,17.7,14.4,7.1,75.0,25.0,46.583333,11.5,2.704167
4,CO,DF,BRASILIA,A001,-15.789444,-47.925833,1159.54,2000-05-07,2014-01-05,0.0,...,11.866667,29.3,16.7,14.2,7.2,79.0,26.0,49.333333,9.0,2.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998900,S,SC,CAMPOS NOVOS,A898,-27.388611,-51.215833,963.00,2019-02-15,2023-12-27,0.0,...,11.400000,27.5,11.9,15.6,7.9,97.0,30.0,62.857143,7.9,3.209524
1998901,S,SC,CAMPOS NOVOS,A898,-27.388611,-51.215833,963.00,2019-02-15,2023-12-28,0.0,...,15.529167,30.1,16.6,18.7,14.4,89.0,40.0,68.208333,11.0,4.220833
1998902,S,SC,CAMPOS NOVOS,A898,-27.388611,-51.215833,963.00,2019-02-15,2023-12-29,2.8,...,16.678261,26.0,16.8,18.5,15.6,96.0,56.0,79.043478,19.6,2.460870
1998903,S,SC,CAMPOS NOVOS,A898,-27.388611,-51.215833,963.00,2019-02-15,2023-12-30,0.2,...,14.113636,25.2,15.6,17.2,10.1,98.0,40.0,73.454545,8.2,1.781818


In [47]:
# saving dataset

dataset.to_csv('./weather_data.csv', index=False)